In [2]:
import netCDF4
import numpy as np
import time
import matplotlib.pyplot as plt
import xarray as xr
import dask

path_to_file = "/fast/gmooers/Real_Geography_Manuscript/Preprocessed_Data/One_Month_Apr/full_physics_essentials_valid_month02_targets.nc"
ds = xr.open_dataset(path_to_file)
truths = ds.targets[:, :30].values
lons = ds.lon.values
lats = ds.lat.values
path_to_file = "/fast/gmooers/Real_Geography_Manuscript/Models/Good_Apr.nc"
ds = xr.open_dataset(path_to_file)
features = ds.Prediction[:, :30].values

x = 144
y = 96
z = 30
t = int(len(truths)/(x*y))

begin
halfway
2879


In [3]:
reconstructed_targets = np.zeros(shape=(x, y, t, z))
reconstructed_features = np.zeros(shape=(x, y, t, z))

count = 0
for i in range(t):
    for j in range(y):
        for k in range(x):
            A = truths[count]
            B = features[count]
            reconstructed_targets[k, j, i, :] = A
            reconstructed_features[k, j, i, :] = B
            count = count + 1

hi
(144, 96, 2879, 30)
(144, 96, 2879, 30)


In [4]:
#turn 15 minute increments into daily averages
ndays = int(t/96)
target_days = np.zeros(shape=(x, y, ndays, z))
feat_days = np.zeros(shape=(x, y, ndays, z))
day_array_targ = np.zeros(shape=(96, z))
day_array_feat = np.zeros(shape=(96, z))
count = 0
day_count = 0
for i in range(x):
    for j in range(y):
        for k in range(t):
            temp_targ = reconstructed_targets[i, j, k, :]
            day_array_targ[count, :] = temp_targ
            temp_feat = reconstructed_features[i, j, k, :]
            day_array_feat[count,:] = temp_feat
            if (count == 95) or (k == t):
                targ = np.nanmean(day_array_targ, axis = 0)
                feat = np.nanmean(day_array_feat, axis = 0)
                target_days[i, j, day_count, :] = targ
                feat_days[i, j, day_count, :] = feat
                day_count = day_count + 1
                count = 0
            else:
                count = count+1
        count = 0
        day_count = 0 

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
(144, 96, 29, 30)
(144, 96, 29, 30)


In [5]:
#code for all longitudes 
A = target_days
B = feat_days
R = np.zeros(shape=(x, y, 1, z))
SSE = np.zeros(shape=(x, y, 1, z))
SVAR = np.zeros(shape=(x, y, 1, z))
target_list = []
feature_list = []
count = 0

for i in range(len(A)):
    for j in range(len(A[0])):
        for k in range(len(A[0][0][0])):
            for l in range(len(A[0][0])):
                target_value = A[i][j][l][k]
                feature_value = B[i][j][l][k]
                target_list.append(target_value)
                feature_list.append(feature_value)
            
            target_list = np.array(target_list)
            feature_list = np.array(feature_list)
            sse = np.sum((target_list-feature_list)**2.0)
            svar = np.sum((target_list-np.mean(target_list))**2.0)
            r_2 = 1-(sse/svar)
            SSE[i, j, :, k] = sse
            SVAR[i, j, :, k] = svar
            R[i, j, :, k] = r_2
            target_list = []
            feature_list = []
    
S = np.nanmean(SSE, axis = 0)
S = np.squeeze(S)
VAR = np.nanmean(SVAR, axis = 0)
VAR = np.squeeze(VAR)

R_Save = np.squeeze(R)
SSE_Save = np.squeeze(SSE)
SVAR_Save = np.squeeze(SVAR)
np.save("/fast/gmooers/Real_Geography_Manscript/Data_For_Paper/April_SPCAM5_SSE_Daily_Interval_Heat.npy", SSE_Save)
np.save("/fast/gmooers/Real_Geography_Manscript/Data_For_Paper/April_SPCAM5_SVAR_Daily_Interval_Heat.npy", SVAR_Save)
np.save("/fast/gmooers/Real_Geography_Manscript/Data_For_Paper/April_SPCAM5_Daily_Interval_Heat.npy", R_Save)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
hi
(144, 96, 1, 30)


In [8]:
y = 64
x = 128
z = 30
tt = 16
#For grid that takes into account levels
#SPCAM5
#datas = netCDF4.Dataset("/fast/gmooers/Raw_Data/extras/TimestepOutput_Neuralnet_SPCAM_216.cam.h1.2009-01-01-72000.nc")
#SPCAM3
datas = netCDF4.Dataset("/fast/gmooers/Real_Geography_Manuscript/Data_For_Paper/AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0001-12-31-00000.nc")

#Open up the variables
lons = np.array(others.variables['lon'])
lats = np.array(others.variables['lat'])
times = np.array(others.variables['time'])
ps = np.array(others.variables['PS'])
#SPCAM5
#pttend = np.array(others.variables['PTTEND'])
#SPCAM3
pttend = np.array(others.variables['TPHYSTND'])
levs = np.array(others.variables['lev'])

p = np.zeros(shape=(x, y, z, tt))
#formula
hyam = np.array(others.variables['hyam'])
hybm = np.array(others.variables['hybm'])

#loop over longitude
for i in range(len(p)):
    #loop over latitude
    for j in range(len(p[0])):
        #loop over time
        for k in range(len(p[0][0][0])):
            p_temp = hyam + ps[k][j][i]*hybm
            p[i,j,:,k] = p_temp/100.
        

#simple mesh grid
Xvar, Yvar = np.meshgrid(lons, levs)
Xs, Ys = np.meshgrid(lats, levs)
ilat = []
for i in range(len(lats)):
    if lats[i] < 10.0 or lats[i] > -10.0:
        ilat.append(lats[i])
        
ilat = np.array(ilat)

#CORRECT WAY:
pband = np.nanmean(p,axis = 3)
pband = np.nanmean(pband,axis = 0)
np.save("/fast/gmooers/Real_Geography_Manscript/Data_For_Paper/SPCAM3_X_Coords.npy", Xs.T)
np.save("/fast/gmooers/Real_Geography_Manscript/Data_For_Paper/SPCAM3_Z_Coords.npy", pband)

(48, 64, 128)
(128, 64, 30, 16)
(128, 64, 30)
(64, 30)
(30, 64)
